# Home Credit Program 1

In [5]:
# Program hc02

# Libraries and settings
import numpy as np
import pandas as pd

import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn import linear_model
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from xgboost import XGBRegressor
import matplotlib.pyplot as plt
# import seaborn as sns

# Use this to turn warnings off (needed for the sklearn regression)
import warnings

import plotly.express as px
import statsmodels.api as sm



In [6]:
pd.set_option('display.max_rows', 1000)
pd.options.display.float_format = "{:,.0f}".format
np.set_printoptions(linewidth=120)
cr="\n\n"

def shrink(x,d,m):
    lower=np.subtract(x,d)
    upper=np.add(x,d)
    mean_below_lower=np.where(m<lower,lower,0)
    mean_above_upper=np.where(m>upper,upper,0)
    combined=mean_below_lower+mean_above_upper
    together=np.where(combined==0,m,combined)
    return(together)

In [9]:
train=pd.read_csv('train.csv')
test=pd.read_csv('test.csv')
train.describe()
print(train.SalePrice.mean())

180921.19589041095


In [35]:
#train.head(100)

In [11]:
print("Full train dataset shape is {}".format(train.shape))
print("Full test dataset shape is {}".format(test.shape))


Full train dataset shape is (1460, 81)
Full test dataset shape is (1459, 80)


In [12]:
# Count number of unique values
input_uniqvals = pd.DataFrame(train.nunique(axis=0),columns=["Unique Values"])
input_uniqvals

,Unique Values
Id,1460
MSSubClass,15
MSZoning,5
LotFrontage,110
LotArea,1073
Street,2
Alley,2
LotShape,4
LandContour,4
Utilities,2


In [13]:
# input_dtypes = pd.DataFrame(train.dtypes, columns=["Data Type"])
# input_dtypes
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1460 entries, 0 to 1459
Data columns (total 81 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Id             1460 non-null   int64  
 1   MSSubClass     1460 non-null   int64  
 2   MSZoning       1460 non-null   object 
 3   LotFrontage    1201 non-null   float64
 4   LotArea        1460 non-null   int64  
 5   Street         1460 non-null   object 
 6   Alley          91 non-null     object 
 7   LotShape       1460 non-null   object 
 8   LandContour    1460 non-null   object 
 9   Utilities      1460 non-null   object 
 10  LotConfig      1460 non-null   object 
 11  LandSlope      1460 non-null   object 
 12  Neighborhood   1460 non-null   object 
 13  Condition1     1460 non-null   object 
 14  Condition2     1460 non-null   object 
 15  BldgType       1460 non-null   object 
 16  HouseStyle     1460 non-null   object 
 17  OverallQual    1460 non-null   int64  
 18  OverallC

In [36]:
#test.info()

In [37]:
numerics=['LotFrontage',
'LotArea',
'OverallQual',
'OverallCond',
'YearBuilt',
'YearRemodAdd',
'MasVnrArea',
'BsmtFinSF1',
'BsmtFinSF2',
'BsmtUnfSF',
'TotalBsmtSF',
'1stFlrSF',
'2ndFlrSF',
'LowQualFinSF',
'GrLivArea',
'BsmtFullBath',
'BsmtHalfBath',
'FullBath',
'HalfBath',
'BedroomAbvGr',
'KitchenAbvGr',
'TotRmsAbvGrd',
'Fireplaces',
'GarageYrBlt',
'GarageCars',
'GarageArea',
'WoodDeckSF',
'OpenPorchSF',
'EnclosedPorch',
'3SsnPorch',
'ScreenPorch',
'PoolArea',
'MiscVal',
'MoSold',
'YrSold'
]
#numerics

In [38]:
objects=['MSZoning',
'Street',
'Alley',
'LotShape',
'LandContour',
'Utilities',
'LotConfig',
'LandSlope',
'Neighborhood',
'Condition1',
'Condition2',
'BldgType',
'HouseStyle',
'RoofStyle',
'RoofMatl',
'Exterior1st',
'Exterior2nd',
'MasVnrType',
'ExterQual',
'ExterCond',
'Foundation',
'BsmtQual',
'BsmtCond',
'BsmtExposure',
'BsmtFinType1',
'BsmtFinType2',
'Heating',
'HeatingQC',
'CentralAir',
'Electrical',
'KitchenQual',
'Functional',
'FireplaceQu',
'GarageType',
'GarageFinish',
'GarageQual',
'GarageCond',
'PavedDrive',
'PoolQC',
'Fence',
'MiscFeature',
'SaleCondition',
'SaleType']

In [39]:
bandable=objects
train1=train.copy()
train1=train1.fillna('')
shrinkage=2.0
averageSP=train1.SalePrice.mean()
for variable in bandable:
    out=train1.groupby(variable).agg(
                         Count=('SalePrice','count'),
                         SalePrice_Mean=('SalePrice', 'mean'),
                         StdError=('SalePrice','sem'))                               
    out['Lower']=np.subtract(out['SalePrice_Mean'],out['StdError'])
    out['Upper']=np.add(out['SalePrice_Mean'],out['StdError'])
    out['Shrunk']=shrink(out['SalePrice_Mean'],shrinkage*out['StdError'],averageSP)
    newname=variable+'B'
    out1=out[["Shrunk"]]
    out1=out1.rename(columns={'Shrunk':newname})
    out=out.rename(columns={'Shrunk':newname})
    print(out,cr)
    train1=train1.join(out1,on=variable)


          Count  SalePrice_Mean  StdError   Lower   Upper  MSZoningB
MSZoning                                                            
C (all)      10          74,528    10,686  63,842  85,214     95,899
FV           65         214,014     6,496 207,518 220,510    201,023
RH           16         131,558     8,929 122,630 140,487    149,415
RL         1151         191,005     2,381 188,624 193,386    186,244
RM          218         126,317     3,286 123,031 129,603    132,889 


        Count  SalePrice_Mean  StdError   Lower   Upper  StreetB
Street                                                          
Grvl        6         130,190    26,718 103,472 156,909  180,921
Pave     1454         181,131     2,083 179,047 183,214  180,921 


       Count  SalePrice_Mean  StdError   Lower   Upper  AlleyB
Alley                                                         
        1369         183,452     2,180 181,272 185,632 180,921
Grvl      50         122,219     4,919 117,300 127,138 132,057

In [18]:
out1

,SaleConditionB
SaleCondition,
Abnorml,"154,765"
AdjLand,"117,193"
Alloca,"180,921"
Family,"160,293"
Normal,"177,216"
Partial,"263,017"


In [19]:
train1.describe()

,Id,MSSubClass,LotArea,OverallQual,OverallCond,YearBuilt,YearRemodAdd,BsmtFinSF1,BsmtFinSF2,BsmtUnfSF,...,GarageTypeB,GarageFinishB,GarageQualB,GarageCondB,PavedDriveB,PoolQCB,FenceB,MiscFeatureB,SaleTypeB,SaleConditionB
count,"1,460","1,460","1,460","1,460","1,460","1,460","1,460","1,460","1,460","1,460",...,"1,460","1,460","1,460","1,460","1,460","1,460","1,460","1,460","1,460","1,460"
mean,730,57,"10,517",6,6,"1,971","1,985",444,47,567,...,"179,766","179,767","179,102","179,350","179,338","180,995","179,968","180,105","182,111","182,643"
std,422,42,"9,981",1,1,30,21,456,161,442,...,"36,344","40,780","20,334","20,790","16,610","2,001","12,036","4,389","25,695","25,496"
min,1,20,"1,300",1,1,"1,872","1,950",0,0,0,...,"106,963","106,963","106,963","106,963","119,714","180,921","140,854","133,000","136,270","117,193"
25%,366,20,"7,554",5,5,"1,954","1,967",0,0,223,...,"136,195","144,047","185,314","185,713","184,258","180,921","185,219","180,921","175,400","177,216"
50%,730,50,"9,478",6,5,"1,973","1,994",384,0,478,...,"200,277","198,976","185,314","185,713","184,258","180,921","185,219","180,921","175,400","177,216"
75%,"1,095",70,"11,602",7,6,"2,000","2,004",712,0,808,...,"200,277","198,976","185,314","185,713","184,258","180,921","185,219","180,921","175,400","177,216"
max,"1,460",190,"215,245",10,9,"2,010","2,010","5,644","1,474","2,336",...,"243,854","234,885","196,050","185,713","184,258","235,000","185,219","180,921","265,599","263,017"


In [20]:
train1[["Fence","FenceB"]]

,Fence,FenceB
0,,"185,219"
1,,"185,219"
2,,"185,219"
3,,"185,219"
4,,"185,219"
...,...,...
1455,,"185,219"
1456,MnPrv,"154,026"
1457,GdPrv,"180,921"
1458,,"185,219"


In [122]:
bandable=['LotFrontage','YearBuilt']
#bandable=['LotFrontage','BldgType']
#bandable=objects
train1=train.copy()
shrinkage=1.0
averageSP=train1.SalePrice.mean()
for variable in bandable:
    train1[variable] = pd.qcut(train1[variable],q=6,precision=2,duplicates='drop')
    out=train1.groupby(variable).agg(
                         Count=('SalePrice','count'),
                         SalePrice_Mean=('SalePrice', 'mean'),
                         StdError=('SalePrice','sem'))                               
    out['Lower']=np.subtract(out['SalePrice_Mean'],out['StdError'])
    out['Upper']=np.add(out['SalePrice_Mean'],out['StdError'])
    out['Shrunk']=shrink(out['SalePrice_Mean'],shrinkage*out['StdError'],averageSP)
    newname=variable+'B'
    out1=out[["Shrunk"]]
    out1=out1.rename(columns={'Shrunk':newname})
    out=out.rename(columns={'Shrunk':newname})
    print(out,cr)
    train1=train1.join(out1,on=variable)


               Count  SalePrice_Mean  StdError   Lower   Upper  LotFrontageB
LotFrontage                                                                 
(20.99, 50.0]    205         152,451     4,626 147,825 157,076       157,076
(50.0, 60.0]     242         137,595     3,449 134,146 141,044       141,044
(60.0, 69.0]     154         182,350     5,765 176,585 188,115       180,921
(69.0, 77.0]     205         175,651     4,270 171,381 179,921       179,921
(77.0, 87.0]     198         203,984     5,160 198,824 209,144       198,824
(87.0, 313.0]    197         244,040     8,293 235,747 252,333       235,747 


                   Count  SalePrice_Mean  StdError   Lower   Upper  YearBuiltB
YearBuilt                                                                     
(1871.99, 1940.0]    257         133,904     3,539 130,365 137,443     137,443
(1940.0, 1960.0]     245         138,493     2,626 135,867 141,119     141,119
(1960.0, 1973.0]     238         147,378     2,770 144,608 150,14

In [158]:
#train1.head()
#train1[['LotFrontage','LotFrontageB']]


In [21]:
print(out)

               Count  SalePrice_Mean  StdError   Lower   Upper  SaleConditionB
SaleCondition                                                                 
Abnorml          101         146,527     8,239 138,288 154,765         154,765
AdjLand            4         104,125    13,068  91,057 117,193         117,193
Alloca            12         167,377    24,382 142,996 191,759         180,921
Family            20         149,600    10,693 138,907 160,293         160,293
Normal          1198         175,202     2,014 173,188 177,216         177,216
Partial          125         272,292     9,275 263,017 281,567         263,017


In [40]:
outp = out.rename_axis('SaleCondition').reset_index()
fig = px.bar(outp, x='SaleCondition', y=['Count'],barmode='group',color_discrete_sequence=['gray'])            
#fig.update_layout(title=dict(text="SaleCondition"))
fig.show()

In [56]:
v='SaleType'
vB=v+"B"
outp = out.rename_axis(v).reset_index()
fig = px.bar(outp, x=v, y=['SalePrice_Mean',vB],barmode='group',color_discrete_sequence=['green','brown'])            
fig.update_layout(title=dict(text="Mean SalePrice by "+v))
fig.show()

In [ ]:
# Testing
'''
#x1=np.array([250,225,200,175,150,125,100])
#d1=np.array([20,10,10,10,5,5,10])
#m1=175
x1=np.array([152141,137595,182350,175651,203984,244040])

d1=np.array([4626,3449,5765,4270,5160,8293])
m1=180000
lower=np.subtract(x1,d1)
upper=np.add(x1,d1)
mean_below_lower=np.where(m1<lower,lower,0)
mean_above_upper=np.where(m1>upper,upper,0)
combined=mean_below_lower+mean_above_upper
together=np.where(combined==0,m1,combined)

#print(shrink(x1,d1,m))
print(cr,x1,cr,d1,cr,lower,cr,upper,cr,cr,mean_below_lower,cr,mean_above_upper,cr,combined,cr,together,cr)
'''

'\n#x1=np.array([250,225,200,175,150,125,100])\n#d1=np.array([20,10,10,10,5,5,10])\n#m1=175\nx1=np.array([152141,137595,182350,175651,203984,244040])\n\nd1=np.array([4626,3449,5765,4270,5160,8293])\nm1=180000\nlower=np.subtract(x1,d1)\nupper=np.add(x1,d1)\nmean_below_lower=np.where(m1<lower,lower,0)\nmean_above_upper=np.where(m1>upper,upper,0)\ncombined=mean_below_lower+mean_above_upper\ntogether=np.where(combined==0,m1,combined)\n\n#print(shrink(x1,d1,m))\nprint(cr,x1,cr,d1,cr,lower,cr,upper,cr,cr,mean_below_lower,cr,mean_above_upper,cr,combined,cr,together,cr)\n'

In [32]:
for i in range(0, len(objects)):
    str = objects[i]
    objects[i] = objects[i]+'B'

#print(objects)

In [33]:
# Perform a Multiple  Linear Regression

# Settings the warnings to be ignored
warnings.filterwarnings('ignore')

#X=train1[['ExterQualB','HeatingQCB',]]
X=train1[objects]
y=train1['SalePrice']
regr = linear_model.LinearRegression()
regr.fit(X, y)
print(regr.coef_)
print(f'r_sqr value: {regr.score(X, y)}')


[ 6.80212386e-02 -1.01666194e+12 -1.10183716e-01  1.21963501e-01  1.30167007e-01  4.35764847e+11  6.39331818e-01
  1.69078949e+11  5.04211426e-01  4.73518372e-02 -1.21364594e-01  5.10025024e-02  3.11355591e-01  5.06946564e-01
  2.63474154e+00  1.46362305e-01 -1.98257446e-01  3.81622314e-02  1.82800293e-01  2.81639099e-02 -1.22650146e-01
  2.26928711e-01  5.99288940e-02  4.68353271e-01  6.22100830e-02 -3.04008484e-01  1.19150162e-01  6.90612793e-02
  5.67169189e-02 -2.79998779e-02  3.14453125e-01  2.59672165e-01  3.75732422e-01  5.75561523e-02  2.44750977e-02
  1.66168213e-02  1.37298584e-01 -7.20825195e-02  4.01990446e+11 -9.33685303e-02 -1.75379515e-01  1.18820190e-01
  1.80511475e-02]
r_sqr value: 0.7549165527473392


In [34]:
train1[['SalePrice','ExterQualB','HeatingQCB']]

,SalePrice,ExterQualB,HeatingQCB
0,208500,"225,188","208,488"
1,181500,"147,163","208,488"
2,223500,"225,188","208,488"
3,140000,"147,163","163,677"
4,250000,"225,188","208,488"
...,...,...,...
1455,175000,"147,163","208,488"
1456,210000,"147,163","146,928"
1457,266500,"335,077","208,488"
1458,142125,"147,163","163,677"


In [161]:
input_dtypes = pd.DataFrame(train1.dtypes, columns=["Data Type"])
input_dtypes

,Data Type
Id,int64
MSSubClass,int64
MSZoning,object
LotFrontage,object
LotArea,int64
Street,object
Alley,object
LotShape,object
LandContour,object
Utilities,object


In [163]:
train1.keys()

Index(['Id', 'MSSubClass', 'MSZoning', 'LotFrontage', 'LotArea', 'Street',
       'Alley', 'LotShape', 'LandContour', 'Utilities',
       ...
       'GarageTypeB', 'GarageFinishB', 'GarageQualB', 'GarageCondB',
       'PavedDriveB', 'PoolQCB', 'FenceB', 'MiscFeatureB', 'SaleTypeB',
       'SaleConditionB'],
      dtype='object', length=124)

In [164]:
train1.index.dtype

dtype('int64')